In [1]:
import pandas as pd

In [11]:
def no_dupes(df):
    
    #creating df of number of times a claim repeats

    repeats = df['ClaimID'].value_counts().to_frame().reset_index()

    #getting list of duplicate claims
    
    all_dupes = repeats[repeats['ClaimID']>1]
    dupes = all_dupes['index'].to_list()
   

    #getting one index value per duplicate claim

    goodi=[]
    for e in dupes:
        goodi.append(min(df[df['ClaimID'].isin([e])].index.to_list()))
    
    # full list of duplicate claim indicies

    alli=[]
    for e in dupes:
        alli = alli + df[df['ClaimID'].isin([e])].index.to_list()


    # take goodi from alli
    badi = [i for i in alli if i not in goodi]

    #drop duplicate indicies
    return(df.drop(index=badi).reset_index().drop(columns='index'))

In [50]:

def aggregate_me(df, by, lg, column_list):
    
    # making df usable

    df[by] = df[by].str.upper()
    
    #filtering for local government

    if lg: 

        lglist = ['Local Government (Counties)',
         'Local Government ()',
         'Local Government (Cities)',
         'Local Government ']

        df = df[df['Grouping'].isin(lglist)]


    # creating a total programs column

    prg = df.groupby([by, 'PrgID'], dropna = False).size().index.tolist() 
    #need to count the number of times a city/county shows up 

    e,f = [list(c) for c in zip(*prg)]

    ccp = pd.Series(e) # made a series so can use value_counts

    ccp = ccp.str.upper()

    tp = ccp.value_counts(dropna = False) #total programs per city
    
    # creating a total claims column
    
    df['Total Claims'] = [1]*len(df)
    
    #aggregate!
    relevant_columns = column_list + ['Total Claims']
    agg = df.groupby(by).sum()[relevant_columns]
    agg['Total Programs'] = tp
    
    # Resource/OBF budget as percentage of total budget
    obf_budget = df.groupby(['SiteCity', 'OBF_Flag']).Budget.sum().unstack().fillna(0).loc[:, True]
    resource_budget = df.groupby(['SiteCity', 'Resource_Flag']).Budget.sum().unstack().fillna(0).loc[:, 1]
    agg['% OBF Budget'] = (obf_budget/agg['Budget'])*100
    agg['% Resource Budget'] = (resource_budget/agg['Budget'])*100
    
    return(agg.reset_index())

In [17]:
k12 = pd.read_csv('data/cpuc_claims_k-12_programs_2017_2019.csv')

k12 = k12.drop_duplicates()

df = no_dupes(k12)

In [52]:
br = pd.read_csv('data/cpuc_with_budget_and_resource.csv', usecols=['ClaimID','Budget', 'Resource_Flag'])

In [53]:
br.head()

,ClaimID,Budget,Resource_Flag
0,PGE-2017-Q4-85848,5541.932761,True
1,PGE-2017-Q4-94241,106.846221,True
2,PGE-2017-Q4-88304,39.779768,True
3,PGE-2017-Q4-8624,1354.671628,True
4,PGE-2017-Q4-88820,106.118823,True


In [54]:
k12br = df.merge(br, left_on='ClaimID', right_on='ClaimID', how='left')

In [28]:
k12br[k12br.Budget.isna()]

,PrgYear,ClaimID,PrgID,ProgramName,PA,Sector,SiteCity,SiteZipCode,SiteID,NAICSCode,...,TotalGrossMeasureCost,TotalGrossMeasureCost_ER,TotalLifecycleGrosskW,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Upstream_Flag,WaterOnly_Flag,OBF_Flag_y,Budget,Resource_Flag
199,2018,PGE-2018-Q4-88219,PGE210143,Hospitality Program,PGE,Com,WALNUT CREEK,94598,PGE-1484044,611113,...,34594.5400,33159.33,0.000000,754354.08,0.0000,False,False,NaN,NaN,NaN
200,2018,PGE-2018-Q4-88218,PGE210143,Hospitality Program,PGE,Com,WALNUT CREEK,94598,PGE-1484043,611113,...,35950.4600,33748.24,0.000000,1215043.20,0.0000,False,False,NaN,NaN,NaN
1564,2017,PGE-2017-Q4-91844,PGE21022,Industrial Deemed Incentives,PGE,Com,NAPA,94558,PGE-1098604,611113,...,47.0600,0.00,0.595056,1898.88,-17.2224,False,False,NaN,NaN,NaN
1624,2017,SCE-2017-Q4-0296409,SCE-13-SW-001B,Plug Load and Appliances Program,SCE,Res,VISALIA,93277,SCE_01044202,611112,...,24.6164,0.00,0.206800,1017.50,0.0000,False,False,NaN,NaN,NaN


## There are four claims that somehow do not exist in the cpuc with budget dataset I created?

In [55]:
x = k12br[k12br.Budget.isna()].ClaimID.to_list()

br[br.ClaimID.isin(x)]

,ClaimID,Budget,Resource_Flag


In [56]:
cpuc = pd.read_csv('data/cpuc_claims_selected_programs_2017_2019.csv', usecols=['ClaimID'])

cpuc[cpuc.ClaimID.isin(x)]

,ClaimID


## Looks like these claims are not even in the big ol original cpuc dataset we have, sorta a mystery where they came from then....
for now will just aggregate without them

In [57]:
nob = k12br[k12br.Budget.isna()].index.to_list()

k_12 = k12br.drop(index=nob)

In [58]:
k_12.columns

Index(['PrgYear', 'ClaimID', 'PrgID', 'ProgramName', 'PA', 'Sector',
       'SiteCity', 'SiteZipCode', 'SiteID', 'NAICSCode', 'BldgHVAC', 'BldgLoc',
       'BldgType', 'BldgVint', 'E3ClimateZone', 'E3GasSavProfile',
       'E3GasSector', 'E3MeaElecEndUseShape', 'E3TargetSector',
       'ImplementationID', 'InstallationDate', 'NAICSBldgType', 'NumUnits',
       'OBF_Flag', 'PrgElement', 'RateScheduleElec', 'RateScheduleGas',
       'REN_Flag', 'Residential_Flag', 'SchoolIdentifier',
       'TotalFirstYearGrosskW', 'TotalFirstYearGrosskWh',
       'TotalFirstYearGrossTherm', 'TotalGrossIncentive',
       'TotalGrossMeasureCost', 'TotalGrossMeasureCost_ER',
       'TotalLifecycleGrosskW', 'TotalLifecycleGrosskWh',
       'TotalLifecycleGrossTherm', 'Upstream_Flag', 'WaterOnly_Flag', 'Budget',
       'Resource_Flag'],
      dtype='object')

In [59]:
column_list = ['TotalFirstYearGrosskWh',
       'TotalFirstYearGrossTherm','TotalLifecycleGrosskWh',
       'TotalLifecycleGrossTherm','Budget']

In [62]:
k12_cities = aggregate_me(k_12, 'SiteCity', False, column_list)

In [63]:
k12_cities.head()

,SiteCity,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Total Claims,Total Programs,% OBF Budget,% Resource Budget
0,AGOURA HILLS,179743.037040,-684.882951,8.987152e+05,-3424.414757,47724.814452,10,1,0.0,100.0
1,ALHAMBRA,189245.340000,0.000000,2.270944e+06,0.000000,108470.812395,4,1,0.0,100.0
2,ALISO VIEJO,169144.476670,-11.348487,2.036566e+06,-136.181842,42661.616682,13,3,0.0,100.0
3,ALTA LOMA,198349.646512,-792.654227,1.056730e+06,-3963.319514,77320.719585,10,2,0.0,100.0
4,ALTADENA,73893.328000,-326.874240,1.108400e+06,-4903.113600,110484.301637,8,1,0.0,100.0


## now gotta add that demographics file

In [97]:
demos = pd.read_csv('data/city_demographics.csv').iloc[:, 1:]

In [98]:
demos.head()

,city,dac_proportion,ces_score_median,ces_percentile_median,Total Tax Revenue,Average Tax Revenue,Population,median_household_income_usd
0,ACAMPO,0.0,21.531463,40.282507,NaN,NaN,NaN,83913.0
1,ACTON,0.0,14.513222,24.063564,NaN,NaN,NaN,105263.0
2,ADELANTO,0.5,35.908799,68.451255,23628867.0,7.876289e+06,34049.0,45695.5
3,AGOURA HILLS,0.0,8.967396,10.827343,58614832.0,1.953828e+07,20222.0,128054.5
4,ALAMEDA,0.0,20.596928,39.796948,282357554.0,9.411918e+07,77624.0,101246.5


In [99]:
demos[demos['city']=='VENTURA']

,city,dac_proportion,ces_score_median,ces_percentile_median,Total Tax Revenue,Average Tax Revenue,Population,median_household_income_usd
738,VENTURA,0.08,19.78211,37.810569,278789652.0,92929884.0,109106.0,78115.0


In [88]:
demos['city'] = demos['city'].str.upper()

In [89]:
k12all = k12_cities.merge(demos, left_on='SiteCity', right_on='city', how='left')

In [90]:
k12all.shape

(192, 18)

In [91]:
k12_cities.shape

(192, 10)

In [92]:
k12all.head()

,SiteCity,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Total Claims,Total Programs,% OBF Budget,% Resource Budget,city,dac_proportion,ces_score_median,ces_percentile_median,Total Tax Revenue,Average Tax Revenue,Population,median_household_income_usd
0,AGOURA HILLS,179743.037040,-684.882951,8.987152e+05,-3424.414757,47724.814452,10,1,0.0,100.0,AGOURA HILLS,0.00,8.967396,10.827343,58614832.0,1.953828e+07,20222.0,128054.5
1,ALHAMBRA,189245.340000,0.000000,2.270944e+06,0.000000,108470.812395,4,1,0.0,100.0,ALHAMBRA,0.15,33.702151,66.143272,154981924.0,5.166064e+07,83750.0,59206.0
2,ALISO VIEJO,169144.476670,-11.348487,2.036566e+06,-136.181842,42661.616682,13,3,0.0,100.0,ALISO VIEJO,0.00,8.231658,9.093202,54563796.0,1.818793e+07,50887.0,108687.0
3,ALTA LOMA,198349.646512,-792.654227,1.056730e+06,-3963.319514,77320.719585,10,2,0.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALTADENA,73893.328000,-326.874240,1.108400e+06,-4903.113600,110484.301637,8,1,0.0,100.0,ALTADENA,0.00,17.007650,30.369530,NaN,NaN,NaN,101875.0


In [94]:
k12all[k12all.SiteCity=='VENTURA']

,SiteCity,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Total Claims,Total Programs,% OBF Budget,% Resource Budget,city,dac_proportion,ces_score_median,ces_percentile_median,Total Tax Revenue,Average Tax Revenue,Population,median_household_income_usd
182,VENTURA,276381.339616,-1225.448007,1.810740e+06,-9287.588036,93647.112696,44,2,0.0,100.0,VENTURA,0.08,19.78211,37.810569,NaN,NaN,NaN,78115.0


In [85]:
nanindex = k12all[k12all.Population.isna()].index